In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json, math

In [ ]:
models = [
    ("<path>/Experiments/peso_med_OctreeNCA2DSegmentation/", "Med-NCA"),
    ("<path>/Experiments/pesofFixAbl_none_10_1.0_16_3_1.0_0.99_OctreeNCA2DSegmentation/", "OctreeNCA"),
    ("<path>/Experiments/peso_UNet_efficientnet-b0_0_MinUNet2DSegmentation/", "Efficientnet"),
    ("<path>/Experiments/peso_UNet_mobilenet_v2_0_MinUNet2DSegmentation/", "Mobilenetv2"),
    ("<path>/Experiments/peso_UNet_resnet18_0_MinUNet2DSegmentation/", "Resnet18"),
    ("<path>/Experiments/peso_UNet_vgg11_0_MinUNet2DSegmentation/", "VGG11"),
    ("<path>/Experiments/peso_UnetPlusPlus_resnet34_0_MinUNet2DSegmentation/", "UNet++"),
    ("<path>/Experiments/peso_unet_UNet2DSegmentation/", "UNet"),
    ("<path>/Experiments/peso_vit_SegFomer2DSegmentation/", "SegFormer"),
]

In [ ]:
annnotation_list = []
annotation_dict = {}
all_files = []
for i, (model, model_name) in enumerate(models):
    score = pd.read_csv(f"{model}/eval/standard.csv", sep='\t').loc[:, "PatchwiseDiceScore/0"] * 100

    score_std = score.std()
    score = score.mean()

    mem_allocation = json.load(open(f"{model}/mem_allocation.json"))["byte"]
    num_params = json.load(open(f"{model}/num_params.json"))["num_params"]

    eval_file = {}

    eval_file["model"] = model
    eval_file["mem_allocation"] = mem_allocation
    eval_file["num_params"] = num_params
    eval_file["score"] = score
    eval_file["score_std"] = score_std
    df = pd.DataFrame(index=[i], data=eval_file)
    all_files.append(df)
    annnotation_list.append((num_params, score, model_name))
    annotation_dict[model_name] = (num_params, score)

all_files = pd.concat(all_files)

In [ ]:
from IPython.display import display, HTML
display(HTML(all_files.to_html()))

In [ ]:
sam_performance = pd.read_csv("<path>/Experiments/peso_sam_Sam2DSegmentation/eval/standard.csv", sep='\t').loc[:, "PatchwiseDiceScore/0"] * 100
sam_performance = sam_performance.mean()

sam_num_params = json.load(open("<path>/Experiments/peso_sam_Sam2DSegmentation/num_params.json"))["num_params"]
annotation_dict["SAM"] = (sam_num_params, sam_performance)

In [ ]:
sns.set_theme()
ax = sns.scatterplot(data=all_files, x="num_params", y="score", hue="mem_allocation", size="mem_allocation",legend=False, sizes=(20, 500))
#ax.annotate('SAM', (sam_num_params, sam_performance), textcoords="offset points", xytext=(0,10), ha='center')
scatter = ax.scatter(sam_num_params, sam_performance, color='green', marker='*', s=375, edgecolor='white', linewidth=1.25)
for x,y,txt in annnotation_list:
    #ax.annotate(txt, (x,y), textcoords="offset points", xytext=(0,10), ha='center')
    pass


def my_annotate(name, xytext):
    return ax.annotate(name, annotation_dict[name], textcoords="offset points", xytext=xytext, ha='center',
        bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white"),
        arrowprops=dict(arrowstyle="->", color='black'))
    
my_annotate("OctreeNCA", (10, 20))
my_annotate("Med-NCA", (30, -30))
my_annotate("UNet++", (60, 30))
my_annotate("Efficientnet", (30, 30)).zorder = 0.9
my_annotate("UNet", (40, -20))
my_annotate("VGG11", (10, -70))
my_annotate("Mobilenetv2", (-40, -75))
my_annotate("Resnet18", (-20, -100))
my_annotate("SegFormer", (-50, -30))
my_annotate("SAM", (-50, 30))


plt.xscale("log")
plt.xlabel("Number of parameters")
plt.ylabel("Dice")
plt.title("Pathology segmentation")
plt.ylim(38.5, 100)
plt.savefig("figures/pathology_segmentation.pdf", bbox_inches='tight')

In [ ]:
min_vram = math.floor(10 * all_files["mem_allocation"].min() / 1024**3) / 10
max_vram = math.ceil(10 * all_files["mem_allocation"].max() / 1024**3) / 10


def compute_circle_size(mem_allocation):
    mem_allocation = mem_allocation / 1024**3
    return 20 + 480 * (mem_allocation - min_vram) / (max_vram - min_vram)

In [ ]:
all_files.loc[:, "circle_size"] = all_files["mem_allocation"].apply(compute_circle_size)

In [ ]:
sns.set_theme()
ax = sns.scatterplot(data=all_files, x="num_params", y="score", hue="mem_allocation", size="mem_allocation",legend=False, sizes=(20, 500))
scatter = ax.scatter(sam_num_params, sam_performance, color='green', marker='*', s=375, edgecolor='white', linewidth=1.25)



def my_annotate(name, xytext):
    return ax.annotate(name, annotation_dict[name], textcoords="offset points", xytext=xytext, ha='center',
        bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white"),
        arrowprops=dict(arrowstyle="->", color='black'))
    
my_annotate("OctreeNCA", (10, 20))
my_annotate("Med-NCA", (30, -30))
my_annotate("UNet++", (60, 30))
my_annotate("Efficientnet", (30, 30)).zorder = 0.9
my_annotate("UNet", (40, -20))
my_annotate("VGG11", (10, -70))
my_annotate("Mobilenetv2", (-40, -75))
my_annotate("Resnet18", (-20, -100))
my_annotate("SegFormer", (-50, -30))
my_annotate("SAM", (-50, 30))


size_values = [20, 500]
size_labels = [f"{min_vram} GiB", f"{max_vram} GiB"]
handles = [
    plt.scatter([], [], s=size, edgecolor="black", color="grey", label=label)
    for size, label in zip(size_values, size_labels)  # Adjust sizes to match your plot
]
# Add the custom legend
plt.legend(
    handles=handles,
    title="Memory Allocation",
    labelspacing=1.2,
    loc="lower left",
    borderpad=1.0,
    frameon=True
)

plt.xscale("log")
plt.xlabel("Number of parameters")
plt.ylabel("Dice")
plt.title("Pathology segmentation")
plt.ylim(38.5, 100)
plt.savefig("figures/pathology_segmentation.pdf", bbox_inches='tight')